In [23]:
import keras
from keras.datasets import cifar10
import numpy as np 
import itertools
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import accuracy_score
from sklearn.neighbors import BallTree
import pickle
import glob
!pip install opencv-contrib-python==4.4.0.44
import cv2
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA

In [24]:

# We are loading cifar10 data from keras
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [25]:
# x_train = x_train[1:5000]
# y_test = y_train[1:5000]
# y_test = y_test[1:5000]
# y_test = y_test[1:5000]

from sklearn.model_selection import train_test_split

# creating matrix
# input_matrix = np.arange(46928*28*28).reshape((46928,28,28))
second_size = 15000/50000

X1, X2 = train_test_split(x_train, test_size=second_size)
Xt1, Xt2 = train_test_split(x_test, test_size=second_size)
Y1, Y2 = train_test_split(y_train, test_size=second_size)
Yt1, Yt2 = train_test_split(y_test, test_size=second_size)

x_train = X2
x_test = Xt2

y_train = Y2
y_test = Yt2
print('X1 shape: ', x_train.shape)
print('X2 shape: ', x_test.shape)
print('X2 shape: ', y_train.shape)
print('X2 shape: ', y_test.shape)

X1 shape:  (15000, 32, 32, 3)
X2 shape:  (3000, 32, 32, 3)
X2 shape:  (15000, 1)
X2 shape:  (3000, 1)


In [26]:
def desSIFT(image):
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(image,None)
    #draw keypoints
    #import matplotlib.pyplot as plt		
    #img2 = cv2.drawKeypoints(img,kp,None,(255,0,0),4)
    #plt.imshow(img2),plt.show()
    return kp,des

def describeORB( image):
    #An efficient alternative to SIFT or SURF
    #doc http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_feature2d/py_orb/py_orb.html
    #ORB is basically a fusion of FAST keypoint detector and BRIEF descriptor 
    #with many modifications to enhance the performance
    orb=cv2.ORB_create()
    kp, des=orb.detectAndCompute(image,None)
    return kp,des

# def describeSURF( image):
#     surf = cv2.xfeatures2d.SURF_create()
#     # it is better to have this value between 300 and 500
#     surf.setHessianThreshold(400)
#     kp, des = surf.detectAndCompute(image,None)
#     return kp,des


In [27]:
def getDescriptors(images) : 
    descriptors = []
    
    for image in images : 
        print (image.shape)
        # Converting the image into gray scale
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
        # Re-sizing the image
        # image = cv2.resize(image, (150, 150), interpolation=cv2.INTER_AREA)
        kp, des = desSIFT(image)
        if des is not None : 
            descriptors.append(des)
            
    descriptors = np.concatenate(descriptors, axis=0)
    descriptors = np.asarray(descriptors)
        
    return descriptors

def getVLADDescriptors(images, images_lables, visualDic):
    descriptors = []
    labels = []
    
    count = 0
    for image in images : 
        #       Converting the image into gray scale
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
        #     Re-sizing the image
        # image = cv2.resize(image, (150, 150), interpolation=cv2.INTER_AREA)
        kp, des = desSIFT(image)
        if des is not None : 
            v = VLAD(des, visualDic)
            descriptors.append(v)
            labels.append(images_lables[count])
        count += 1
            
            
    descriptors = np.asarray(descriptors)
    print(labels)
    return descriptors, labels

def VLAD(X, visualDictionary) : 
    
    predictedLabels = visualDictionary.predict(X)
    centers = visualDictionary.cluster_centers_
    labels = visualDictionary.labels_
    k = visualDictionary.n_clusters
    
    m,d = X.shape
    V=np.zeros([k,d])
    #computing the differences

    # for all the clusters (visual words)
    for i in range(k):
        # if there is at least one descriptor in that cluster
        if np.sum(predictedLabels==i)>0:
            # add the diferences
            V[i]=np.sum(X[predictedLabels==i,:]-centers[i],axis=0)
    

    V = V.flatten()
    # power normalization, also called square-rooting normalization
    V = np.sign(V)*np.sqrt(np.abs(V))

    # L2 normalization

    V = V/np.sqrt(np.dot(V,V))
    return V

In [ ]:
surf_des = getDescriptors(np.concatenate((x_train, x_test), axis = 0))

In [29]:
k = 100
visDic = MiniBatchKMeans(init='k-means++', n_clusters=50,max_iter=1000, batch_size=1000, n_init=10, max_no_improvement=10, verbose=0).fit(surf_des)

In [30]:
vlad_des, labels = getVLADDescriptors(x_train, y_train, visDic)
# getting vlad descriptors on the test data
vlad_des_test, labels_test = getVLADDescriptors(x_test, y_test, visDic)
print  (vlad_des_test)
print (labels_test)

[array([3], dtype=uint8), array([9], dtype=uint8), array([8], dtype=uint8), array([8], dtype=uint8), array([8], dtype=uint8), array([4], dtype=uint8), array([4], dtype=uint8), array([9], dtype=uint8), array([7], dtype=uint8), array([3], dtype=uint8), array([1], dtype=uint8), array([0], dtype=uint8), array([3], dtype=uint8), array([9], dtype=uint8), array([2], dtype=uint8), array([6], dtype=uint8), array([0], dtype=uint8), array([9], dtype=uint8), array([3], dtype=uint8), array([8], dtype=uint8), array([0], dtype=uint8), array([9], dtype=uint8), array([6], dtype=uint8), array([6], dtype=uint8), array([0], dtype=uint8), array([6], dtype=uint8), array([0], dtype=uint8), array([4], dtype=uint8), array([0], dtype=uint8), array([9], dtype=uint8), array([4], dtype=uint8), array([0], dtype=uint8), array([1], dtype=uint8), array([1], dtype=uint8), array([9], dtype=uint8), array([3], dtype=uint8), array([3], dtype=uint8), array([3], dtype=uint8), array([2], dtype=uint8), array([1], dtype=uint8),

In [31]:
sc = StandardScaler() 
pca = PCA(n_components = 0.9) 

vlad_des = sc.fit_transform(vlad_des) 
vlad_des_test = sc.transform(vlad_des_test) 
vlad_des = pca.fit_transform(vlad_des) 
vlad_des_test = pca.transform(vlad_des_test)

In [32]:
clf = cv2.ml.KNearest_create()
clf.train(np.float32(vlad_des), cv2.ml.ROW_SAMPLE, np.float32(labels))

True

In [33]:
ret, results, neighbours ,dist = clf.findNearest(np.float32(vlad_des_test), k=10)

In [34]:
pred_label = []
correct_label = []

for label in results:
    pred_label.append(int(label))

for label in labels_test:
    correct_label.append(int(label))
    
accuracy_score(labels_test, pred_label)

2993
2993


0.09421984630805212